<a href="https://colab.research.google.com/github/MGentieu/Data_analytics_aircraft_messages/blob/main/GENTIEU_Martin_TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GENTIEU Martin : TP de data analytics

###On se place dans un premier temps dans le répertoire parent

In [41]:
import os
import sys
from pathlib import Path

# Detect working directory
cwd = Path.cwd().resolve()

# If in Colab (/content), search for repo folder automatically
if cwd.name == "content":
    # Look for the cloned project automatically
    candidates = [d for d in cwd.iterdir() if d.is_dir() and "aircraft" in d.name.lower()]

    if not candidates:
        raise FileNotFoundError(
            f"Aucun dossier contenant 'aircraft' trouvé dans /content.\n"
            f"Dossiers présents : {os.listdir(cwd)}"
        )

    PROJECT_ROOT = candidates[0]
else:
    PROJECT_ROOT = cwd

print(f"Detected project root: {PROJECT_ROOT}")

# Validate directory
if not PROJECT_ROOT.exists():
    raise FileNotFoundError(f"Missing directory at {PROJECT_ROOT}")

# Add to Python path
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("Setup complete.")


Detected project root: /content/Data_analytics_aircraft_messages
Setup complete.


On importe ensuite les bibliothèques nécessaires à l'analyse :

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import folium
import shapely
import socket
import csv
from datetime import datetime

On exécute ensuite le script python test.py pour récupérer les messages

In [56]:
HOST = "sbs.glidernet.org"
PORT = 30003

# Timestamp pour nommer le fichier
date = datetime.now().strftime('%Y%m%d_%H%M%S')

# Fichier CSV de sortie
output_file = f"{PROJECT_ROOT}/adsb_data_{date}.csv"

# Liste des colonnes SBS (22 champs)
cols = [
    "MessageType",
    "TransmissionType",
    "SessionID",
    "AircraftID",
    "HexIdent",
    "FlightID",
    "DateGenerated",
    "TimeGenerated",
    "DateLogged",
    "TimeLogged",
    "Callsign",
    "Altitude",
    "GroundSpeed",
    "Track",
    "Latitude",
    "Longitude",
    "VerticalRate",
    "Squawk",
    "Alert",
    "Emergency",
    "SPI",
    "IsOnGround"
]

print(f"Connexion à {HOST}:{PORT} ...")

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    print("Connecté ! Réception des messages ADS-B...\n")

    with open(output_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Message"])  # Une seule colonne pour l'instant

        try:
            for i in range(500):  # Nombre de messages à lire
                data = s.recv(1024).decode(errors="ignore")
                for line in data.strip().split("\n"):
                    if line.startswith("MSG"):
                        writer.writerow([line])
                        print(line)
        except KeyboardInterrupt:
            print("\nArrêt manuel.")
        except Exception as e:
            print("Erreur :", e)

print(f"\nDonnées enregistrées dans : {output_file}")


Connexion à sbs.glidernet.org:30003 ...
Connecté ! Réception des messages ADS-B...


Arrêt manuel.

Données enregistrées dans : /content/Data_analytics_aircraft_messages/adsb_data_20251119_114449.csv


###1.1: Chargement du fichier avec la librairie pandas

In [48]:
cols = [
    "MessageType",
    "TransmissionType",
    "SessionID",
    "AircraftID",
    "HexIdent",
    "FlightID",
    "DateGenerated",
    "TimeGenerated",
    "DateLogged",
    "TimeLogged",
    "Callsign",
    "Altitude",
    "GroundSpeed",
    "Track",
    "Latitude",
    "Longitude",
    "VerticalRate",
    "Squawk",
    "Alert",
    "Emergency",
    "SPI",
    "IsOnGround"
]


df = pd.read_csv(output_file)
#df = df[0].str.split(",", expand=True)

KeyError: 0

Analyse initiale du dataset :

In [45]:
df.head()

,MessageType,TransmissionType,SessionID,AircraftID,HexIdent,FlightID,DateGenerated,TimeGenerated,DateLogged,TimeLogged,...,GroundSpeed,Track,Latitude,Longitude,VerticalRate,Squawk,Alert,Emergency,SPI,IsOnGround
0,Message,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"MSG,3,,,494141,,2025/11/19,11:32:46.372,2025/1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"MSG,3,,,4B9068,,2025/11/19,11:32:46.378,2025/1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"MSG,3,,,449825,,2025/11/19,11:32:46.380,2025/1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"MSG,3,,,043EA0,,2025/11/19,11:32:46.382,2025/1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3793 entries, 0 to 3792
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MessageType       3793 non-null   object 
 1   TransmissionType  0 non-null      float64
 2   SessionID         0 non-null      float64
 3   AircraftID        0 non-null      float64
 4   HexIdent          0 non-null      float64
 5   FlightID          0 non-null      float64
 6   DateGenerated     0 non-null      float64
 7   TimeGenerated     0 non-null      float64
 8   DateLogged        0 non-null      float64
 9   TimeLogged        0 non-null      float64
 10  Callsign          0 non-null      float64
 11  Altitude          0 non-null      float64
 12  GroundSpeed       0 non-null      float64
 13  Track             0 non-null      float64
 14  Latitude          0 non-null      float64
 15  Longitude         0 non-null      float64
 16  VerticalRate      0 non-null      float64


In [47]:
df.describe()

,TransmissionType,SessionID,AircraftID,HexIdent,FlightID,DateGenerated,TimeGenerated,DateLogged,TimeLogged,Callsign,...,GroundSpeed,Track,Latitude,Longitude,VerticalRate,Squawk,Alert,Emergency,SPI,IsOnGround
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
